In [ ]:
# For Google Colab to access Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
PATH = '.' #'/content/drive/My Drive/TFG-Image-Optimization' # for Google Colab
INPATH=PATH+'border/inputs/'
SIGN_PATH=INPATH+'fuji-20018-x6.jpg'
BICYCLE_PATH=INPATH+'sony-20177-x12.jpg'

In [ ]:
# David Cabezas Berrido

# border.ipynb

# Examples of approaches to solve the shape problem

import matplotlib.pyplot as plt
import cv2

import tensorflow as tf

GEN_PATH=PATH+'models/GAN-generator'
OUPATH=PATH+'border/outputs/'
OUPATH0=OUPATH'border-0/'
OUPATHCOPY=OUPATH+'border-copy/'

generator = tf.keras.models.load_model(GEN_PATH)

def decode_normalize(img_path):
    img = tf.cast(tf.image.decode_jpeg(tf.io.read_file(img_path)),tf.float32)[...,:3]
    img = img/127.5-1
    return img

def lightUp0(img_path): # Zero padding
    dat = tf.data.Dataset.from_tensor_slices([img_path])
    dat = dat.map(decode_normalize)
    dat = dat.batch(1)
    for img in dat:
        HEIGHT = img.shape[1] # Zero padding so the the height and width are divisible by 256
        WIDTH = img.shape[2]
        HPAD = (256-HEIGHT%256)%256 
        WPAD = (256-WIDTH%256)%256
        img = tf.keras.layers.ZeroPadding2D(padding=((0,HPAD),(0,WPAD)))(img)
        prediction = generator(img, training=True) # Prediction for input
        
    out = prediction[0]
    out = out[:HEIGHT,:WIDTH]
    return out*0.5+0.5

def lightUpCopy(img_path): # Border replication
    dat = tf.data.Dataset.from_tensor_slices([img_path])
    dat = dat.map(decode_normalize)
    dat = dat.batch(1)
    for img in dat:
        HEIGHT = img.shape[1]
        WIDTH = img.shape[2]
        HPAD = (256-HEIGHT%256)%256
        WPAD = (256-WIDTH%256)%256
        # Horizontal bottom border
        row = img[0,-1,...]
        hpad = tf.tile(row[tf.newaxis,...],[HPAD,1,1])[tf.newaxis,...]
        img = tf.keras.layers.Concatenate(axis=1)([img,hpad])
        # Vertical right border
        col = img[0,:,-1,...]
        wpad = tf.tile(col[:,tf.newaxis,...],[1,WPAD,1])[tf.newaxis,...]
        img = tf.keras.layers.Concatenate(axis=2)([img,wpad])
        prediction = generator(img, training=True) # Prediction for input
        
    out = prediction[0]
    out = out[:HEIGHT,:WIDTH]
    return out*0.5+0.5

In [ ]:
sign=plt.imread(SIGN_PATH) # Save the image in different shapes
img=cv2.resize(sign, dsize=(1536,1024), interpolation=cv2.INTER_AREA)
plt.imsave(INPATH+'sign/1024x1536.jpg',img)
img=cv2.resize(sign, dsize=(1400,900), interpolation=cv2.INTER_AREA)
plt.imsave(INPATH+'sign/900x1400.jpg',img)

In [ ]:
path=plt.imread(BICYCLE_PATH) # Save the image in different shapes
img=cv2.resize(path, dsize=(1536,1024), interpolation=cv2.INTER_AREA)
plt.imsave(INPATH+'bicycle/1024x1536.jpg',img)
img=cv2.resize(path, dsize=(1400,900), interpolation=cv2.INTER_AREA)
plt.imsave(INPATH+'bicycle/900x1400.jpg',img)

Generate predictions for different shapes.

Zero-padding.

In [ ]:
pred=lightUp0(INPATH+'bicycle/1024x1536.jpg')
plt.imshow(pred)
plt.show()
tf.keras.preprocessing.image.save_img(OUPATH0+'bicycle1024x1536.jpg',pred) # Save to disc

In [ ]:
pred=lightUp0(INPATH+'sign/1024x1536.jpg')
plt.imshow(pred)
plt.show()
tf.keras.preprocessing.image.save_img(OUPATH0+'sign1024x1536.jpg',pred) # Save to disc

In [ ]:
pred=lightUp0(INPATH+'bicycle/900x1400.jpg')
plt.imshow(pred)
plt.show()
tf.keras.preprocessing.image.save_img(OUPATH+'bicycle900x1400.jpg',pred) # Save to disc

In [ ]:
pred=lightUp0(INPATH+'sign/900x1400.jpg')
plt.imshow(pred)
plt.show()
tf.keras.preprocessing.image.save_img(OUPATH0+'sign900x1400.jpg',pred) # Save to disc

Border replication.

In [ ]:
pred=lightUpCopy(INPATH+'bicycle/1024x1536.jpg')
plt.imshow(pred)
plt.show()
tf.keras.preprocessing.image.save_img(OUPATHCOPY+'bicycle1024x1536.jpg',pred) # Save to disc

In [ ]:
pred=lightUpCopy(INPATH+'sign/1024x1536.jpg')
plt.imshow(pred)
plt.show()
tf.keras.preprocessing.image.save_img(OUPATHCOPY+'sign1024x1536.jpg',pred) # Save to disc

In [ ]:
pred=lightUpCopy(INPATH+'bicycle/900x1400.jpg')
plt.imshow(pred)
plt.show()
tf.keras.preprocessing.image.save_img(OUPATHCOPY+'bicycle900x1400.jpg',pred) # Save to disc

In [ ]:
pred=lightUpCopy(INPATH+'sign/900x1400.jpg')
plt.imshow(pred)
plt.show()
tf.keras.preprocessing.image.save_img(OUPATHCOPY+'sign900x1400.jpg',pred) # Save to disc